In [1]:
from config import *
from utilities import get_model_name
from dataset import ClassificationDataset

import functools
import numpy as np
import pandas as pd

results_dict = {}
internal, external = 11, 12
encode_method = "dummy"
impute_method = "mean"
fs_method, fs_ratio = "dnp", 0.5
norm_method = "maxmin"
classification_dataset = ClassificationDataset(
    internal, external, encode_method, impute_method, fs_method, fs_ratio, norm_method, random_state=SEED)

In [2]:
# from sklearn.neural_network import MLPClassifier
from utilities import MLPClassifier

model_fn = functools.partial(
    MLPClassifier, random_state=SEED, max_iter=50,
    momentum=0.9, squares=0.999, optimizer_t='sgdm',
    use_residual=True, use_batch_norm=False)
param_grid_list = [{
    # hidden_layer_sizes = (64, 32)
    # config       lr_range            worst_auc
    # adam w/ bn:  [2e-5, 5e-5, 1e-4]  0.8942
    # adam w/o bn: [5e-5, 1e-4, 2e-4]  0.9148
    # sgdm w/ bn:  [1e-3, 2e-3, 5e-3]  0.8982
    # sgdm w/o bn: [5e-3, 1e-2, 2e-2]  0.9177

    'hidden_layer_sizes': [(128, 128), (64, 64, 64)],
    'learning_rate': [5e-3, 1e-2, 2e-2],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

MLPClassifier


Epoch 1/50:   0%|          | 0/26 [00:00<?, ?batch/s]

model(hidden_layer_sizes=(128, 128), learning_rate=0.005)
Valid:         AUC: 0.9178 | AUPR: 0.6613 | Acc: 95.38%


model(hidden_layer_sizes=(128, 128), learning_rate=0.01)
Valid:         AUC: 0.9209 | AUPR: 0.6694 | Acc: 95.41%


model(hidden_layer_sizes=(128, 128), learning_rate=0.02)
Valid:         AUC: 0.9230 | AUPR: 0.6728 | Acc: 95.35%


model(hidden_layer_sizes=(64, 64, 64), learning_rate=0.005)
Valid:         AUC: 0.9174 | AUPR: 0.6583 | Acc: 95.36%


model(hidden_layer_sizes=(64, 64, 64), learning_rate=0.01)
Valid:         AUC: 0.9220 | AUPR: 0.6672 | Acc: 95.44%


model(hidden_layer_sizes=(64, 64, 64), learning_rate=0.02)
Valid:         AUC: 0.9231 | AUPR: 0.6702 | Acc: 95.46%
best model(hidden_layer_sizes=(64, 64, 64), learning_rate=0.02)
Valid:         AUC: 0.9231 | AUPR: 0.6702 | Acc: 95.46%
Internal test: AUC: 0.9320 | AUPR: 0.7021 | Acc: 95.18%
External test: AUC: 0.9235 | AUPR: 0.6710 | Acc: 95.20%


In [3]:
from sklearn.svm import SVC

model_fn = functools.partial(
    SVC, random_state=SEED, probability=True,
    class_weight='balanced')
param_grid_list = [{
    'kernel': ['linear'],
    'C': [0.01, 0.1, 1],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

SVC
model(kernel=linear, C=0.01)
Valid:         AUC: 0.9176 | AUPR: 0.6250 | Acc: 95.04%
model(kernel=linear, C=0.1)
Valid:         AUC: 0.9180 | AUPR: 0.6185 | Acc: 95.04%
model(kernel=linear, C=1)
Valid:         AUC: 0.9172 | AUPR: 0.6143 | Acc: 95.01%
best model(kernel=linear, C=0.1)
Valid:         AUC: 0.9180 | AUPR: 0.6185 | Acc: 95.04%
Internal test: AUC: 0.9260 | AUPR: 0.6748 | Acc: 95.06%
External test: AUC: 0.9166 | AUPR: 0.6165 | Acc: 94.70%


In [4]:
from sklearn.neighbors import KNeighborsClassifier

model_fn = functools.partial(
    KNeighborsClassifier, n_jobs=-1, algorithm='auto', p=2)
param_grid_list = [{
    'n_neighbors': [3, 5, 7],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

KNeighborsClassifier
model(n_neighbors=3)
Valid:         AUC: 0.7545 | AUPR: 0.4238 | Acc: 94.70%
model(n_neighbors=5)
Valid:         AUC: 0.7874 | AUPR: 0.4764 | Acc: 95.04%
model(n_neighbors=7)
Valid:         AUC: 0.8029 | AUPR: 0.4963 | Acc: 95.04%
best model(n_neighbors=7)
Valid:         AUC: 0.8029 | AUPR: 0.4963 | Acc: 95.04%
Internal test: AUC: 0.8011 | AUPR: 0.5307 | Acc: 94.96%
External test: AUC: 0.8115 | AUPR: 0.5041 | Acc: 94.75%


In [5]:
from sklearn.naive_bayes import GaussianNB

model_fn = GaussianNB
param_grid_list = [{}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

GaussianNB
model()
Valid:         AUC: 0.8745 | AUPR: 0.4371 | Acc: 82.96%
best model()
Valid:         AUC: 0.8745 | AUPR: 0.4371 | Acc: 82.96%
Internal test: AUC: 0.8817 | AUPR: 0.4701 | Acc: 83.55%
External test: AUC: 0.8710 | AUPR: 0.4185 | Acc: 81.98%


In [6]:
# from sklearn.exceptions import ConvergenceWarning
# import warnings
# warnings.filterwarnings("ignore", category=ConvergenceWarning)
from sklearn.linear_model import LogisticRegression

model_fn = functools.partial(
    LogisticRegression, random_state=SEED, n_jobs=-1, 
    max_iter=2000, solver='saga', penalty='l2',
    )
param_grid_list = [{
    'C': [1.0, 10.0, 100.0],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(
    model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(
    best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

LogisticRegression


model(C=1.0)
Valid:         AUC: 0.9192 | AUPR: 0.6560 | Acc: 95.27%
model(C=10.0)
Valid:         AUC: 0.9191 | AUPR: 0.6551 | Acc: 95.27%
model(C=100.0)
Valid:         AUC: 0.9190 | AUPR: 0.6548 | Acc: 95.25%
best model(C=1.0)
Valid:         AUC: 0.9192 | AUPR: 0.6560 | Acc: 95.27%
Internal test: AUC: 0.9273 | AUPR: 0.6896 | Acc: 95.09%
External test: AUC: 0.9152 | AUPR: 0.6372 | Acc: 94.96%


In [7]:
from sklearn.tree import DecisionTreeClassifier

model_fn = functools.partial(
    DecisionTreeClassifier, random_state=SEED,
    ccp_alpha=0.0, class_weight='balanced', max_depth=5)
param_grid_list = [{
        'criterion': ['gini', 'log_loss', 'entropy'],
    }
]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

DecisionTreeClassifier


model(criterion=gini)
Valid:         AUC: 0.8660 | AUPR: 0.5576 | Acc: 82.66%
model(criterion=log_loss)
Valid:         AUC: 0.8680 | AUPR: 0.5729 | Acc: 82.98%
model(criterion=entropy)
Valid:         AUC: 0.8680 | AUPR: 0.5729 | Acc: 82.98%
best model(criterion=log_loss)
Valid:         AUC: 0.8680 | AUPR: 0.5729 | Acc: 82.98%
Internal test: AUC: 0.8716 | AUPR: 0.5926 | Acc: 79.39%
External test: AUC: 0.8814 | AUPR: 0.5961 | Acc: 78.72%


In [8]:
from sklearn.ensemble import RandomForestClassifier

model_fn = functools.partial(
    RandomForestClassifier, random_state=SEED, n_jobs=-1,
    ccp_alpha=0.0, class_weight='balanced', max_depth=5, bootstrap=False)
param_grid_list = [{
        'criterion': ['gini', 'log_loss', 'entropy'],
        'n_estimators': [50, 100, 200],
    }
]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

RandomForestClassifier
model(criterion=gini, n_estimators=50)
Valid:         AUC: 0.9127 | AUPR: 0.6534 | Acc: 88.99%
model(criterion=gini, n_estimators=100)
Valid:         AUC: 0.9118 | AUPR: 0.6526 | Acc: 89.06%
model(criterion=gini, n_estimators=200)
Valid:         AUC: 0.9123 | AUPR: 0.6539 | Acc: 89.21%
model(criterion=log_loss, n_estimators=50)
Valid:         AUC: 0.9131 | AUPR: 0.6557 | Acc: 88.91%
model(criterion=log_loss, n_estimators=100)
Valid:         AUC: 0.9123 | AUPR: 0.6547 | Acc: 89.06%
model(criterion=log_loss, n_estimators=200)
Valid:         AUC: 0.9124 | AUPR: 0.6556 | Acc: 89.22%
model(criterion=entropy, n_estimators=50)
Valid:         AUC: 0.9131 | AUPR: 0.6557 | Acc: 88.91%
model(criterion=entropy, n_estimators=100)
Valid:         AUC: 0.9123 | AUPR: 0.6547 | Acc: 89.06%
model(criterion=entropy, n_estimators=200)
Valid:         AUC: 0.9124 | AUPR: 0.6556 | Acc: 89.22%
best model(criterion=log_loss, n_estimators=50)
Valid:         AUC: 0.9131 | AUPR: 0.6557 | Acc

In [9]:
from sklearn.ensemble import GradientBoostingClassifier

model = functools.partial(
    GradientBoostingClassifier, random_state=SEED,
    learning_rate=0.1, max_depth=5, loss='log_loss', 
    n_estimators=100)
param_grid_list = [{
        'criterion': ['friedman_mse', 'squared_error'],
    }
]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

GradientBoostingClassifier


model(criterion=friedman_mse)
Valid:         AUC: 0.9200 | AUPR: 0.6575 | Acc: 95.28%
model(criterion=squared_error)
Valid:         AUC: 0.9202 | AUPR: 0.6569 | Acc: 95.27%
best model(criterion=squared_error)
Valid:         AUC: 0.9202 | AUPR: 0.6569 | Acc: 95.27%
Internal test: AUC: 0.9249 | AUPR: 0.6974 | Acc: 95.52%
External test: AUC: 0.9238 | AUPR: 0.6763 | Acc: 95.31%


In [10]:
from sklearn.ensemble import AdaBoostClassifier
model = functools.partial(AdaBoostClassifier, random_state=SEED, algorithm="SAMME")
param_grid_list = [{
    'n_estimators': [100, 200, 500],
    'learning_rate': [1.0],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

AdaBoostClassifier


model(n_estimators=100, learning_rate=1.0)
Valid:         AUC: 0.9182 | AUPR: 0.6380 | Acc: 95.20%
model(n_estimators=200, learning_rate=1.0)
Valid:         AUC: 0.9176 | AUPR: 0.6365 | Acc: 95.13%
model(n_estimators=500, learning_rate=1.0)
Valid:         AUC: 0.9171 | AUPR: 0.6358 | Acc: 95.08%
best model(n_estimators=100, learning_rate=1.0)
Valid:         AUC: 0.9182 | AUPR: 0.6380 | Acc: 95.20%
Internal test: AUC: 0.9207 | AUPR: 0.6747 | Acc: 95.23%
External test: AUC: 0.9131 | AUPR: 0.6181 | Acc: 94.76%


In [11]:
dataframe = pd.DataFrame(results_dict).transpose().reset_index()
dataframe.columns = ['model', 'val_auc', 'val_aupr', 'val_acc', 'intest_auc', 'intest_aupr', 'intest_acc', 'extest_auc', 'extest_aupr', 'extest_acc']
# dataframe.to_csv()
filename = os.path.join(RESULTS_DIR, encode_method, impute_method, f'{fs_method}{int(fs_ratio * 142)}.csv')
dirname = os.path.dirname(filename)
if not os.path.exists(dirname):
    os.makedirs(dirname)
dataframe.to_csv(filename, index=False)